The purpose of this module is to handle interactions with a database.

Reading, writing, creating, deleting, connecting and more.

Goal 1: connect to google

In [4]:
import os
from pathlib import Path

import yaml

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pygsheets

In [2]:
class Timers():
    """A class for  Timing-stamping cell calls."""
    import datetime as dt

    def exec_time(msg="Completed task"):
        """
        Runtime message tracking cell progress. Prints an message and a timestamp.
        
        Parameters
        -------
        msg (str): User provided message. Defaults to a generic statement.
        
        Returns
        -------
        None
        """
        try:
            now = dt.datetime.now().strftime("%H:%M:%S - %Y-%m-%d")
            print(
                "{msg} Timestamp: {now}".format(msg=msg, now=now)
            )
        except Exception as e:
            print("Warning: unable to Run exec_time.\nRawmessage: {msg}.\n{error}".format(msg=msg, error=e))


class Google():
    """"A class to connect to Google services."""

    import pickle
    import pandas as pd
    SERVICE_ACCOUNT = None

    def google_connect(self, credentials_path=None, service_account_env_var=None):
        """
        Connects to google drive and spreadsheets. Requires '[...]/client_secrets[...].json" and or 
        a service account variable in the form of a name (str).
        Will create a token in '.' to track authentication. 
        Returns a service object to allow connections to google drive files.
        Warning: do not share your token or anyone will have access to all content on your drive.

        Parameters
        -------
        credentials_path (str): Optional. Path to client secrets json. Defaults to None.
        service_account_env_var (str): Optional. Name of environment variable for google connection. Defaults to None.  
        
        Returns
        -------
        gdrive (googleapiclient.discovery.Resource object): Resource object with connection to google drive.
        gsheets (pygsheets client object): pygsheets client object to manipulate gsheets.
        """
        
        SCOPES = ["https://www.googleapis.com/auth/drive"]
        gdrive, gsheets = None, None

        if credentials_path != None: 
            creds = None 

            # Authentication flow.
            if Path("token.pickle").exists():
                with open("token.pickle", "rb") as token:
                    creds = self.pickle.load(token)
            if not creds or not creds.valid:
                if creds and creds.expired and creds.refresh_token:
                    creds.refresh(Request())
                else:
                    flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
                    creds = flow.run_local_server(port=0)
                    # Save access token for future use.
                    with open("token.pickle", "wb") as token:
                        self.pickle.dump(creds, token)

            gdrive = build("drive", "v3", credentials=creds)
            gsheets = pygsheets.authorize(custom_credentials=creds)

        elif service_account_env_var != None:
            # dev note: not getting gdrive in this case yet.
            gsheets = pygsheets.authorize(service_account_env_var=service_account_env_var)

        return gdrive, gsheets
    

    def write_to_googlesheets(self, gsheetkey: str, data: pd.DataFrame, wks_title: str, set_df_start="A2", credentials_path=None) -> None:
        """
        Push DataFrame to Googlesheet via key.

        Parameters
        -------
        gsheetkey (str): Key to google sheet.
        data (pd.DataFrame): Dataframe with data to push.
        wks_title (str): Worksheet title.
        set_df_start (str): Defaults to A2. Set where the dataframe starting cell will write. Use A2 formatting.
        credentials_path (str): Filepath to local credentials files. Defaults to None.

        Returns
        -------
        (None)
        """
        df0 = data.copy(deep=True) 
        if credentials_path != None:
            _, gsheets = self.google_connect(credentials_path=credentials_path)
        else:
            _, gsheets = self.google_connect(service_account_env_var=SERVICE_ACCOUNT)

        sh = gsheets.open_by_key(gsheetkey)

        wks = sh.worksheet("title", wks_title)
        wks.clear(start="A1", end=None)

        if wks.rows < len(df0):
            msg = "Warning: Data rows exceeds worksheet rows available. Expanding worksheet."
            logger.warning(msg)
            Timers.exec_time(msg)

            wks.resize(rows=len(df0))

        wks.set_dataframe(df0, start=set_df_start, copy_head=True)

        log_msg = f"Pushed data to gsheet with key:{gsheetkey}"
        logger.info(log_msg)
        Timers.exec_time(log_msg)


def get_response(url: str, username=None, password=None, api_key=None):
    """
    Get a response from API using HTTP.

    Parameters
    -------
    url (str): Url for API request.
    username (str): Optional. Username.
    password (str): Optional. Password.
    api_key (str): API key.

    Returns
    -------
    api_response (requests.models.Response): Response.
    """
    api_response = None
    api_response = requests.get(url, auth=(username, password))
    return api_response

In [47]:
# args
credentials_path = r"/Users/jaimemerizalde/Desktop/JOBS 2023/software/jmailer/secrets/db_secret.json"

db_identifier = "1t1wGAQvZuwEWOOgcgtBaqbZoafG_ZCfTV5QGyMfYHTg"

table_identifier = "contacts"

#filepath or list.
recipients = [
    "marco.starger@getgarner.com", 
    "austin.lovell@getgarner.com", 
    "evelyn.siu@getgarner.com",
]



In [53]:

# db connectivity
# to be specified in an argparse config file for convenience

# Google connectivity
gg = Google()
_, gsheets = gg.google_connect(credentials_path)

In [49]:
# database fetcher
sh = gsheets.open_by_key(db_identifier)


In [50]:
# schema fetcher
worksheets = sh.worksheets()
# titles = [wk.title for wk in worksheets]
#wks_dict = dict(zip(titles, worksheets))
#wks_dfs = dict(zip(titles, [wk.get_as_df() for wk in worksheets]))

In [51]:
# table fetcher
wks = sh.worksheet("title", table_identifier)

In [ ]:
# completes the connection steps.

In [59]:
#  table data  to dataframe
wks.get_as_df()


gaierror: [Errno 8] nodename nor servname provided, or not known

In [ ]:
# the next step is to....

# collect recipient data using cla

# basically, we should have a table-updateer "script" 

# this class or method (originally considered writing a script) is responsible for updating a table provided the data we need.
# credentials as well. 